In [1]:
!pip install cv2
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import os
import cv2

# Define constants
DATA_DIR = r"C:\Users\vaibh\OneDrive\Desktop\predict\sample\archive"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Load annotations
def load_annotations(csv_path, img_dir):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Annotation file not found: {csv_path}")
    
    df = pd.read_csv(csv_path)
    images = []
    labels = []
    
    for _, row in df.iterrows():
        img_path = os.path.join(img_dir, row['filename'])
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, IMG_SIZE)
            images.append(img)
            labels.append(row['class'])
        else:
            print(f"Warning: Image file not found: {img_path}")
    
    if len(images) == 0:
        raise ValueError(f"No images found in {img_dir}")
    
    return np.array(images), np.array(labels)

# Load dataset
def load_dataset(split_name, folder_name):
    csv_path = os.path.join(DATA_DIR, folder_name, '_annotations.csv')
    img_dir = os.path.join(DATA_DIR, folder_name)
    return load_annotations(csv_path, img_dir)

# Ensure correct folder names
data_splits = {'train': 'train', 'test': 'test', 'val': 'valid'}
train_images, train_labels = load_dataset('train', data_splits['train'])
test_images, test_labels = load_dataset('test', data_splits['test'])
val_images, val_labels = load_dataset('val', data_splits['val'])

# Convert labels to numerical values
label_dict = {label: i for i, label in enumerate(set(train_labels))}
train_labels = np.array([label_dict[label] for label in train_labels])
test_labels = np.array([label_dict[label] for label in test_labels])
val_labels = np.array([label_dict[label] for label in val_labels])

# Normalize images
train_images, test_images, val_images = train_images / 255.0, test_images / 255.0, val_images / 255.0

# Define CNN model
model = keras.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_dict), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels), batch_size=BATCH_SIZE)

# Evaluate model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc:.2f}")

model.save("dental_cnn_model.keras")

c:\Users\vaibh\anaconda3\envs\new_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.6457 - loss: 0.9861 - val_accuracy: 0.6923 - val_loss: 0.8730
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6501 - loss: 0.9145 - val_accuracy: 0.6846 - val_loss: 0.8059
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6898 - loss: 0.7832 - val_accuracy: 0.6923 - val_loss: 0.7921
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7113 - loss: 0.7125 - val_accuracy: 0.7115 - val_loss: 0.7426
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7255 - loss: 0.6748 - val_accuracy: 0.6987 - val_loss: 0.7953
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7413 - loss: 0.6355 - val_accuracy: 0.7064 - val_loss: 0.7712
Epoch 7/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7334 - loss: 0.6363 - val_accuracy: 0.7128 - val_loss: 0.7955
Epoch 8/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.7360 - loss: 0.6169 - val_accu